<a href="https://colab.research.google.com/github/aquibjaved/Image-search/blob/master/Copy_of_Image_search_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing PyDrive to access google drive 
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Now, you have to authorize Google SDK to access Google Drive from Colab. 
# First, execute the following commands:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# https://drive.google.com/open?id=1hwoeqVwnNLbL9WGKFl_R9vBKHzKWn98C
# Download trained auto encoders model
# Downloading the file
download = drive.CreateFile({'id': '1hwoeqVwnNLbL9WGKFl_R9vBKHzKWn98C'})
download.GetContentFile('data.h5')

In [0]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import numpy as np
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Input, Conv2D, LSTM, MaxPool2D, UpSampling2D,BatchNormalization
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from numpy import argmax, array_equal
import matplotlib.pyplot as plt
from keras.models import Model
from random import randint

In [0]:
# Laoding model
from keras.models import load_model
model = load_model('./data.h5')
model.summary()

In [7]:
# Load the model, only upto encoder part
encoder = Model(model.layers[0].input, model.layers[9].output)
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
__________

In [0]:
data_train = pd.read_csv("/content/fashionMnist/data/fashion-mnist_train.csv")
data_test = pd.read_csv("/content/fashionMnist/data/fashion-mnist_test.csv")
# Labels 
labels_map = {  0: "T-shirt/top",
                1: "Trouser",
                2: "Pullover",
                3: "Dress",
                4: "Coat",
                5: "Sandal",
                6: "Shirt",
                7: "Sneaker",
                8: "Bag",
                9: "Ankle boot" }

In [0]:
## recreate the train_x array and val_x array
train_x = data_train[list(data_train.columns)[1:]].values
# train_x, val_x = train_test_split(train_x, test_size=0.2)
val_x = data_test[list(data_test.columns)[1:]]
## normalize and reshape
train_x = np.array(train_x/255.)
val_x = np.array(val_x/255.)

train_x = train_x.reshape(-1, 28, 28, 1)
val_x = val_x.reshape(-1, 28, 28, 1)

# Generate the latentent vector for all the training data 
latent_vector_of_training_images = []
labels = data_train[list(data_train.columns)[:1]].values

for img in train_x:
  predicted_vector = encoder.predict(np.expand_dims(img,0))
  reshaped_vect = predicted_vector.reshape(1, 256)
#   print(reshaped_vect[0])  
  latent_vector_of_training_images.append(reshaped_vect[0])

In [14]:
from sklearn.cluster import KMeans
fx = KMeans(n_clusters = 10) 
fx.fit(latent_vector_of_training_images)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [16]:
fx.predict([latent_vector_of_training_images[0]])

array([8], dtype=int32)

In [18]:
fx.

array([[ 1.56585659, -0.63631974,  2.12169488, ...,  0.26309057,
        -0.48502442, -0.46357546],
       [ 1.61155709,  1.15093557,  0.78066459, ..., -0.56488877,
         0.53833103,  0.47528701],
       [ 3.9657465 ,  0.04829047,  2.5655901 , ..., -0.66066247,
        -0.01883831,  0.69185897],
       ...,
       [ 2.74954679,  2.38702055,  1.29501333, ..., -0.31223116,
         1.49152888,  1.03920241],
       [ 0.83761175,  1.52365065,  1.59291994, ...,  0.17429191,
        -0.41095277, -0.35993434],
       [ 1.94445524,  0.19406013,  2.24455483, ...,  0.03924394,
         1.50217712,  1.18036401]])